In [1]:
using Random
using Distributions
using LinearAlgebra
using DataFrames
using StatsBase
using PDMats
using CSV
using Dates
using JuMP
using Ipopt
import SpecialFunctions: loggamma
import MathOptInterface as MOI
using Printf

# Seed (student number)
Random.seed!(23249534)

TaskLocalRNG()

In [2]:
const EPS = 1e-8
safe_beta(α, β) = Beta(max(α, EPS), max(β, EPS))

const LOG_EPS = 1e-200
safe_log(x) = log(max(x, LOG_EPS))

const GAMMA_EPS = 1e-8
function safe_gamma(shape, rate)
    α = max(shape, GAMMA_EPS)
    r = max(rate,  GAMMA_EPS)
    θ = 1/r
    θ = max(θ, GAMMA_EPS)
    return Gamma(α, θ)
end

safe_bernoulli(p::Real) = Bernoulli(clamp(p, 0.0, 1.0))

function safe_pdmat(A::AbstractMatrix{<:Real}; eps0=1e-8, maxiter=5)
    S = Symmetric((A + A')/2)
    for i in 0:maxiter
        try
            return PDMat(Float64.(S + (eps0*i)*I))
        catch e
            if !(e isa PosDefException) || i==maxiter
                rethrow(e)
            end
        end
    end
end

const DIR_EPS = 1e-8
function safe_dirichlet(α::AbstractVector{<:Real})
    a = max.(Float64.(α), DIR_EPS)
    s = sum(a)
    x = rand(Dirichlet(a))
    if any(!isfinite, x) || any(<(0.0), x)
        return a ./ s
    end
    sx = sum(x)
    return (isfinite(sx) && abs(sx - 1.0) ≤ 1e-10) ? x : (a ./ s)
end

safe_dirichlet (generic function with 1 method)

In [3]:
# Load the CSV
df = CSV.read("Y_sim.csv", DataFrame)
select!(df, Not(1))

# Convert to a Matrix{Float64}
dropmissing!(df)
Y = Matrix{Float64}(df)

2515×4 Matrix{Float64}:
  0.0155093     0.0049855    -0.0218385   -0.0093593
 -0.00984817   -0.0310449    -0.0175496   -0.013831
  0.00504508   -0.0259294    -0.0279254   -0.00187619
 -0.00702787    0.00152593    0.00732729   0.0528399
  0.0236889     0.0223468    -0.0103914    0.00394691
 -0.0132638    -0.0173873    -0.0232177   -0.00805076
 -0.00729312   -0.00859467   -0.00334454   0.00981518
 -0.000576131   0.000169942   0.00299624   0.00420534
  0.0054774    -0.0345001    -0.00131918   0.00372307
 -0.000286928  -0.0320366    -0.0429878   -0.0138154
  ⋮                                       
 -0.000487734  -0.00509217   -0.020057    -0.0162265
 -0.0178649    -0.0335209    -0.0264117   -0.0401034
 -0.00937628    0.0112429    -0.0213002    0.0141509
  0.0134912     0.0199176    -0.0173684    0.0270361
  0.037147      0.00332487   -0.00786391  -0.0279697
 -0.000766154   0.0164437     0.0245054    0.0128638
 -0.00192279    0.00342535   -0.00276792   0.017714
 -0.0138138    -0.00810238  

In [4]:
# Number of assets and time horizon
D = size(Y, 2)
T = size(Y, 1)
H = 20

# Number of clusters and MCMC settings
K_init   = 4
N_warmup  = 10000
N_sample  = 50000

# Stick‐breaking and concentration priors
G1, G2 = 1, 5
C1, C2 = 3, 1
X1, X2 = 3, 2

# NIW prior hyperparameters
mu0    = zeros(D)
kappa0  = 5
rho0    = D + 2
psi0_c  = 0.0005
Psi0 = safe_pdmat(psi0_c * Matrix{Float64}(I, D, D))

4×4 PDMat{Float64, Matrix{Float64}}:
 0.0005  0.0     0.0     0.0
 0.0     0.0005  0.0     0.0
 0.0     0.0     0.0005  0.0
 0.0     0.0     0.0     0.0005

In [5]:
function run_pp(Y::Matrix{Float64}, T::Int, H::Int, D::Int)   
    # Set K, δ and S
    K = 4
    δ = Matrix{Float64}(I, K, K)
    S = rand(1:K, T)

    π_true = [ [ Vector{Float64}(undef, K) for _ in 1:K ] for _ in 1:N_sample ]
    μ_true = [ [ Vector{Float64}(undef, D) for _ in 1:K ] for _ in 1:N_sample ]
    Σ_true = [ [ PDMat(Matrix{Float64}(I, D, D)) for _ in 1:K ] for _ in 1:N_sample ]

    # Draw β
    β = safe_dirichlet(ones(K));   β .= β ./ sum(β)

    # Preallocate containers
    ξ = Vector{Float64}(undef, K)
    χ = Vector{Float64}(undef, K)
    α = Vector{Float64}(undef, K)
    λ = Vector{Float64}(undef, K)
    π_vec = Vector{Vector{Float64}}(undef, K)
    μ = Vector{Vector{Float64}}(undef, K)
    Σ = Vector{PDMat{Float64}}(undef, K)

    # Loop over components j = 1:K
    for j in 1:K
        ξ[j] = rand(safe_beta(X1, X2))
        χ[j] = rand(safe_gamma(C1, C2))

        # Compute concentration parameters
        α[j] = (1 - ξ[j]) * χ[j]
        λ[j] =        ξ[j]  * χ[j]

        # Build the Dirichlet parameter vector for π_j
        δ_j = δ[:, j]
        dir_pars = α[j] .* β .+ λ[j] .* δ_j
        π_vec[j] = safe_dirichlet(dir_pars)
        π_vec[j] .= π_vec[j] ./ sum(π_vec[j])

        # Draw from NIW
        Σ_mat = rand(InverseWishart(rho0, Psi0))
        Σ[j]  = safe_pdmat(Σ_mat)
        μ[j]  = rand(MvNormal(mu0, Σ[j] / kappa0))
    end

    # Count assignments
    m     = zeros(Int, K)
    n_mat = zeros(Int, K, K)
    q_mat = zeros(Int, K, K)
    Q     = zeros(Int, K)

    m[S[1]] += 1
    for t in 2:T
        m[S[t]]             += 1
        n_mat[S[t-1], S[t]] += 1
    end

    # Total number of iterations
    total_iters = N_warmup + N_sample
    
    for n_iter in 1:total_iters
        # Update latent states & counts
        for t in 1:T
            # Decrement counts for the current state
            m[S[t]] -= 1
            if t > 1
                n_mat[S[t-1], S[t]] -= 1
            end
            if t < T
                n_mat[S[t], S[t+1]] -= 1
            end
            logW = fill(-Inf, length(β))
            for k in 1:length(β)
                if t == 1
                    log_w_prev = safe_log(β[k]) + safe_log(π_vec[k][S[t+1]])
                elseif t == T
                    log_w_prev = safe_log(π_vec[S[t-1]][k])
                else
                    log_w_prev = safe_log(π_vec[S[t-1]][k]) + safe_log(π_vec[k][S[t+1]])
                end
                if isfinite(log_w_prev)
                    resid = Y[t, :] .- μ[k]
                    quad = dot(resid, (Σ[k]) \ resid)
                    logW[k] = log_w_prev - 0.5 * (quad + logdet(Σ[k]))
                end
            end
            # Stabilise and exponentiate
            maxLW = maximum(logW)
            if !isfinite(maxLW)
                P = β
            else
                W = exp.(logW .- maxLW)
                P = W ./ sum(W)
            end

            # Resample the state
            S[t] = rand(Categorical(P))
            m[S[t]] += 1
            if t > 1
                n_mat[S[t-1], S[t]] += 1
            end
            if t < T
                n_mat[S[t], S[t+1]] += 1
            end
        end

        # Update (μ, Σ)
        for k in 1:K
            if m[k] > 0
                idx    = findall(==(k), S)
                Y_k    = Y[idx, :]
                Y_bar   = vec(sum(Y_k, dims=1)) ./ m[k]
                diff_mu = Y_bar .- mu0
                centered = Y_k .- Y_bar'
                S_k      = centered' * centered
                
                mu_k    = (kappa0*mu0 .+ m[k]*Y_bar) ./ (kappa0 + m[k])
                kappa_k = kappa0 + m[k]
                rho_k   = rho0   + m[k]
                Psi_k   = safe_pdmat(Psi0 + S_k +
                        (kappa0*m[k])/(kappa0+m[k]) * (diff_mu * diff_mu'))
                
                Σ_mat   = rand(InverseWishart(rho_k, Psi_k))
                Σ[k]    = safe_pdmat(Σ_mat)
                μ[k]    = rand(MvNormal(mu_k, Σ[k] / kappa_k))
            else
                Σ_mat = rand(InverseWishart(rho0, Psi0))
                Σ[k]  = safe_pdmat(Σ_mat)
                μ[k]  = rand(MvNormal(mu0, Σ[k] / kappa0))
            end    
        end
        
        # Form CRP variables
        for k in 1:K
            for j in 1:K
                if n_mat[j, k] == 0
                    q_mat[j, k] = 0
                else
                    base = α[j] * β[k] + λ[j] * δ[j, k]
                    b     = zeros(Int, n_mat[j, k])
                    b[1]  = 1
                    for l in 2:n_mat[j, k]
                        b[l] = rand(safe_bernoulli(base / (base + l - 1)))
                    end
                    q_mat[j, k] = sum(b)
                end
            end
        end
        q_alpha_diag  = zeros(Int, K)
        q_lambda_diag = zeros(Int, K)
        for k in 1:K
            qkk = q_mat[k, k]
            if qkk > 0
                a = α[k]*β[k]; b = λ[k]
                denom = a + b
                pα = denom > 0 ? clamp(a/denom, 0.0, 1.0) : 0.5
                q_alpha_diag[k]  = rand(Binomial(qkk, pα))
                q_lambda_diag[k] = qkk - q_alpha_diag[k]
            end
        end
        for k in 1:K
            Q[k] = (sum(q_mat[:, k]) - q_mat[k, k]) + q_alpha_diag[k]
        end
        Q[S[1]] += 1
        M = sum(Q)

        # Update β
        β = safe_dirichlet(ones(K) .+ Q)
        β .= β ./ sum(β)

        # Update ξ and χ
        for j in 1:K
            if m[j] > 0
                ξ[j] = rand(safe_beta(X1 + q_lambda_diag[j], X2 + sum(q_mat[j, :]) - q_lambda_diag[j]))
                r_j  = rand(safe_beta(χ[j] + 1, sum(n_mat[j, :])))
                numerator = C1 + sum(q_mat[j, :]) - 1
                denominator = numerator + sum(n_mat[j, :]) * (C2 - safe_log(r_j))
                z_j  = rand(safe_bernoulli(numerator/denominator))
                χ[j] = rand(safe_gamma(numerator + z_j, C2 - safe_log(r_j)))
                α[j] = (1 - ξ[j]) * χ[j]
                λ[j] = ξ[j]  * χ[j]
            else
                ξ[k] = rand(safe_beta(X1, X2))
                χ[k] = rand(safe_gamma(C1, C2))
                α[k] = (1 - ξ[k]) * χ[k]
                λ[k] = ξ[k]  * χ[k]
            end
        end
        
        # Update π
        for j in 1:K
            π_vec[j] = safe_dirichlet(α[j] .* β .+ λ[j] .* δ[j, :] .+ n_mat[j, :])
            π_vec[j] .= π_vec[j] ./ sum(π_vec[j])
        end 
        
        if n_iter > N_warmup
            s = n_iter - N_warmup

            @inbounds for k in 1:K
                # μ
                if length(μ_true[s][k]) != length(μ[k])
                    μ_true[s][k] = similar(μ[k])
                end
                copyto!(μ_true[s][k], μ[k])
        
                # Σ
                Σ_true[s][k] = Σ[k]
        
                # π
                if length(π_true[s][k]) != length(π_vec[k])
                    π_true[s][k] = similar(π_vec[k])
                end
                copyto!(π_true[s][k], π_vec[k])
            end
        end
    end
    return π_true, μ_true, Σ_true
end

run_pp (generic function with 1 method)

In [6]:
π_true, μ_true, Σ_true = run_pp(Matrix(Y), T, H, D)

K = K_init

π_mean = [ zeros(Float64, K) for _ in 1:K ]
for s in 1:N_sample, k in 1:K
    @inbounds π_mean[k] .+= π_true[s][k]
end
for k in 1:K
    π_mean[k] ./= N_sample
end

# Mean of μ_true
μ_mean = [ zeros(Float64, length(μ_true[1][k])) for k in 1:K ]
for s in 1:N_sample, k in 1:K
    @inbounds μ_mean[k] .+= μ_true[s][k]
end
for k in 1:K
    μ_mean[k] ./= N_sample
end

# Mean of Σ_true
Σ_mean_mat = [ zeros(Float64, size(Matrix(Σ_true[1][k]))...) for k in 1:K ]
for s in 1:N_sample, k in 1:K
    @inbounds Σ_mean_mat[k] .+= Matrix(Σ_true[s][k])
end
for k in 1:K
    Σ_mean_mat[k] ./= N_sample
end

# Symmetrise for numerical safety
eps = 1e-10
Σ_mean = [ PDMat(Symmetric((Σ_mean_mat[k] + Σ_mean_mat[k]')/2 + eps*I)) for k in 1:K ]

floatstr(x) = @sprintf("%.16g", x)

function mat_literal_multiline(M::AbstractMatrix; indent="  ")
    rows = [join(floatstr.(M[i, :]), " ") for i in 1:size(M, 1)]
    return "[" * "\n" * indent * join(rows, ";\n" * indent) * "\n]"
end

function vec_of_vecs_literal_multiline(V::AbstractVector{<:AbstractVector}; indent="  ")
    inner = ["vec([" * join(floatstr.(V[k]), " ") * "])" for k in 1:length(V)]
    return "[" * "\n" * indent * join(inner, ",\n" * indent) * "\n]"
end

function mat_vec_literal_multiline(Mats::AbstractVector{<:AbstractMatrix};
                                   indent="  ", inner_indent="      ")
    parts = [mat_literal_multiline(M; indent=inner_indent) for M in Mats]
    return "[" * "\n" * indent * join(parts, ",\n" * indent) * "\n]"
end

Σ_mats = [Matrix(Σ_mean[k]) for k in 1:length(Σ_mean)]

# Print Values
println("pi_true = ",    vec_of_vecs_literal_multiline(π_mean))
println("\nmu_true = ",    vec_of_vecs_literal_multiline(μ_mean))
println("\nSigma_true = ", mat_vec_literal_multiline(Σ_mats))

pi_true = [
  vec([0.5731372264896774 0.3584768992248216 0.01085525821402486 0.05753061607147426]),
  vec([0.0370388230815912 0.889042514556095 0.002007726412896501 0.07191093594941617]),
  vec([0.006318513965246314 0.0071812424323565 0.7291813565982903 0.257318887004104]),
  vec([0.01157929936688799 0.08442420898922519 0.1134931812666756 0.7905033103772089])
]



mu_true = [
  vec([0.006734506602107449 0.003381498351761594 0.01251958486816306 0.007574169914426993]),
  vec([0.0009441833395542966 0.0003530895525309008 -0.001122619477144013 3.041564764696133e-06]),
  vec([-0.0001357129572322458 0.0001789316615930574 -0.0002532572436555386 -0.0005170677905046179]),
  vec([0.001248987177806565 0.001216024923786339 0.0009936349596729377 -0.0002090026527542731])
]

Sigma_true = [
  [
      0.002054506019709262 0.0005749811760326436 0.0005816549429042377 0.0006146245962203936;
      0.0005749811760326436 0.002379851613164497 0.001639530029524073 0.003591993629918802;
      0.0005816549429042377 0.001639530029524073 0.003565602256146811 0.002759543733341633;
      0.0006146245962203936 0.003591993629918802 0.002759543733341633 0.01157320665447896
],
  [
      0.0002717377294919257 5.902118513846123e-05 6.848107986651424e-05 2.509832559253089e-05;
      5.902118513846123e-05 0.0002444336297386139 0.0001685020755143676 0.0001883782584247908;
      6.848